In [1]:
import joblib
from sklearn.linear_model import LogisticRegression
import pandas as pd


In [2]:
vectorizer = joblib.load("../artifacts/vectorizer.joblib")
df_train = pd.read_csv("../resources/df_train_preprocessed.csv")
df_validation = pd.read_csv("../resources/df_val_preprocessed.csv")
df_test = pd.read_csv("../resources/df_test_preprocessed.csv")

Loading the fitted vectorizer and the preprocessed dataset from 1_eda_preprocessing.ipynb

In [3]:

X_train_vec = vectorizer.transform(df_train["combined_text"])
X_val_vec = vectorizer.transform(df_validation["combined_text"])

# Train model
clf = LogisticRegression(max_iter=1000, class_weight="balanced")
clf.fit(X_train_vec, df_train["medical_specialty"])

# Save final model pipeline (vectorizer + classifier)
from sklearn.pipeline import Pipeline
final_pipeline = Pipeline([
    ('tfidf', vectorizer),
    ('clf', clf)
])



The model gets trained here and an artifact is created from the model. 
- Model was evaluated thoroughly before settling down with this version.

In [4]:
# Evaluate Validation
val_acc = final_pipeline.score(
    df_validation["combined_text"], 
    df_validation["medical_specialty"]
)

# Evaluate Test
test_acc = final_pipeline.score(
    df_test["combined_text"], 
    df_test["medical_specialty"]
)

print(f"📊 Validation Accuracy: {val_acc:.4f}")
print(f"📊 Test Accuracy: {test_acc:.4f}")


📊 Validation Accuracy: 0.8243
📊 Test Accuracy: 0.8135


Here the accuracy ratings for the model is shown. 
- This 0.8243 showed the best potential withing the given time period therefore is selected as the model for deploying.

In [5]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import cross_val_score, GridSearchCV


# -----------------------------
# Step 1: Define Pipeline
# -----------------------------
class_weights = {
    "Orthopedic": 2.0,
    "Neurology": 1.5,
    "Neurosurgery": 1.5,
    # others default to 1.0
}

pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(stop_words='english', lowercase=True)),
    ('clf', LogisticRegression(
        multi_class='multinomial',  # Softmax regression
        solver='lbfgs',
        max_iter=1000,
        class_weight=class_weights
    ))
])

# -----------------------------
# Step 2: Cross-Validation
# -----------------------------
cv_scores = cross_val_score(
    pipeline, 
    df_train['combined_text'], 
    df_train['medical_specialty'], 
    cv=5, 
    scoring='accuracy'
)
print(f"📊 Cross-Validation Accuracy: {cv_scores.mean():.4f} (+/- {cv_scores.std():.4f})")

# -----------------------------
# Step 3: Hyperparameter Tuning (GridSearch)
# -----------------------------
param_grid = {
    'tfidf__ngram_range': [(1,1), (1,2)],
    'tfidf__max_df': [0.8, 0.9, 1.0],
    'tfidf__min_df': [2, 5, 10],
    'tfidf__max_features': [5000, 10000, 20000],
    'clf__C': [0.01, 0.1, 1, 10]
}

grid = GridSearchCV(
    pipeline, 
    param_grid, 
    cv=3, 
    scoring='recall_macro',  # recall_macro helps with rare classes
    n_jobs=-1, 
    verbose=2
)

grid.fit(df_train['combined_text'], df_train['medical_specialty'])

print("✅ Best Parameters:", grid.best_params_)
print(f"📊 Best CV Score: {grid.best_score_:.4f}")

# -----------------------------
# Step 4: Evaluate on Validation & Test Sets
# -----------------------------
best_model = grid.best_estimator_

val_acc = best_model.score(
    df_validation['combined_text'], 
    df_validation['medical_specialty']
)
test_acc = best_model.score(
    df_test['combined_text'], 
    df_test['medical_specialty']
)

print(f"📊 Validation Accuracy (best model): {val_acc:.4f}")
print(f"📊 Test Accuracy (best model): {test_acc:.4f}")

# -----------------------------
# Step 5: Save Best Model
# -----------------------------
joblib.dump(best_model, "../artifacts/model.joblib")
print("💾 Model saved to ../artifacts/model.joblib")

/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/linear_model/_logistic.py:1272: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/linear_model/_logistic.py:1272: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/linear_model/_logistic.py:1272: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
/Library/Frameworks/Pytho

📊 Cross-Validation Accuracy: 0.8167 (+/- 0.0140)
Fitting 3 folds for each of 216 candidates, totalling 648 fits


/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/linear_model/_logistic.py:1272: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/linear_model/_logistic.py:1272: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/linear_model/_logistic.py:1272: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
/Library/Frameworks/Pytho

[CV] END clf__C=0.01, tfidf__max_df=0.8, tfidf__max_features=5000, tfidf__min_df=2, tfidf__ngram_range=(1, 1); total time=   0.6s
[CV] END clf__C=0.01, tfidf__max_df=0.8, tfidf__max_features=5000, tfidf__min_df=5, tfidf__ngram_range=(1, 1); total time=   0.6s
[CV] END clf__C=0.01, tfidf__max_df=0.8, tfidf__max_features=5000, tfidf__min_df=2, tfidf__ngram_range=(1, 1); total time=   0.7s
[CV] END clf__C=0.01, tfidf__max_df=0.8, tfidf__max_features=5000, tfidf__min_df=2, tfidf__ngram_range=(1, 1); total time=   0.7s
[CV] END clf__C=0.01, tfidf__max_df=0.8, tfidf__max_features=5000, tfidf__min_df=5, tfidf__ngram_range=(1, 1); total time=   0.8s


/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/linear_model/_logistic.py:1272: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/linear_model/_logistic.py:1272: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/linear_model/_logistic.py:1272: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
/Library/Frameworks/Pytho

...

[CV] END clf__C=10, tfidf__max_df=1.0, tfidf__max_features=20000, tfidf__min_df=2, tfidf__ngram_range=(1, 2); total time=   4.6s
[CV] END clf__C=10, tfidf__max_df=1.0, tfidf__max_features=20000, tfidf__min_df=10, tfidf__ngram_range=(1, 1); total time=   1.3s
[CV] END clf__C=10, tfidf__max_df=1.0, tfidf__max_features=20000, tfidf__min_df=2, tfidf__ngram_range=(1, 2); total time=   4.5s
[CV] END clf__C=10, tfidf__max_df=1.0, tfidf__max_features=20000, tfidf__min_df=5, tfidf__ngram_range=(1, 2); total time=   4.0s
[CV] END clf__C=10, tfidf__max_df=1.0, tfidf__max_features=20000, tfidf__min_df=5, tfidf__ngram_range=(1, 2); total time=   4.2s


/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/linear_model/_logistic.py:1272: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/linear_model/_logistic.py:1272: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/linear_model/_logistic.py:1272: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


[CV] END clf__C=10, tfidf__max_df=1.0, tfidf__max_features=20000, tfidf__min_df=5, tfidf__ngram_range=(1, 2); total time=   3.7s
[CV] END clf__C=10, tfidf__max_df=1.0, tfidf__max_features=20000, tfidf__min_df=10, tfidf__ngram_range=(1, 2); total time=   2.3s
[CV] END clf__C=10, tfidf__max_df=1.0, tfidf__max_features=20000, tfidf__min_df=10, tfidf__ngram_range=(1, 2); total time=   2.2s
[CV] END clf__C=10, tfidf__max_df=1.0, tfidf__max_features=20000, tfidf__min_df=10, tfidf__ngram_range=(1, 2); total time=   1.9s


/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/linear_model/_logistic.py:1272: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


✅ Best Parameters: {'clf__C': 10, 'tfidf__max_df': 0.9, 'tfidf__max_features': 5000, 'tfidf__min_df': 10, 'tfidf__ngram_range': (1, 1)}
📊 Best CV Score: 0.7902
📊 Validation Accuracy (best model): 0.8459
📊 Test Accuracy (best model): 0.8324
💾 Model saved to ../artifacts/model.joblib


In [6]:
from google.cloud import aiplatform

# Deplyment Details
PROJECT_ID = "fabled-access-470516-r6"
LOCATION = "us-central1"
ENDPOINT_ID = "924991644657254400"

aiplatform.init(project=PROJECT_ID, location=LOCATION)
endpoint = aiplatform.Endpoint(endpoint_name=ENDPOINT_ID)

In [ ]:
# Custom input for prediction
new_symptoms = "The patient presents with headaches, numbness or tingling, muscle weakness, coordination problems, vision changes, speech difficulties, memory issues, confusion, tremors, seizures, and mood changes"

# Get Prediction
response = endpoint.predict (instances=[new_symptoms])

# Print response
print ("Prediction response:", response)
print("Medical Specialty:", response.predictions[0])

Prediction response: Prediction(predictions=['Neurology'], deployed_model_id='6412869683166314496', metadata=None, model_version_id='1', model_resource_name='projects/878540501572/locations/us-central1/models/4406238972242558976', explanations=None)
Medical Specialty: Neurology


## Step 1: Define the Pipeline

In this step, we create a Scikit-learn Pipeline that combines:

- TF-IDF Vectorizer to convert text into numerical features.

- Logistic Regression (multinomial/softmax) as the classifier.

We also assign class weights to handle imbalance, giving more importance to rare classes like Orthopedic, Neurology, and Neurosurgery.

## Step 2 : Cross Validation 

We perform 5-fold cross-validation to estimate the generalization performance of the pipeline.
The metric used is accuracy.

## Step 3 : Hyperparameter Tuning

We run a GridSearchCV to optimize both the TF-IDF parameters and the Logistic Regression regularization parameter C.
The scoring metric is macro recall to emphasize performance on smaller classes.

## Step 4 : Final Evaluation

We evaluate the best model found during GridSearch on both the validation and test sets.

## Step 5 : Save the selected model

Finally, we save the best model pipeline (vectorizer + classifier) into the /artifacts directory for deployment in Vertex AI.

## Step 6 : Deployment On Vertex AI

This created a REST API that can serve real-time predictions. The modal can now be accessed and used for inference directly from cloud.